# Load and preprocess Twenty Newsgroups data

Many of the exercises use email data from the <i>twenty newsgroups</i> data set. More information on the <i>Twenty Newsgroups</i> dataset can be found on the [UCI website](http://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html). This notebook loads that data set and does preprocessing that is a prerequisite for most of the examples in this repository. 

## Setup database connectivity

We'll reuse our module from the previous notebook (***`00_database_connectivity_setup.ipynb`***) to establish connectivity to the database

In [1]:
%run '00_database_connectivity_setup.ipynb'
%matplotlib inline
from IPython.display import display
from IPython.display import HTML

/Users/shajek/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Your connection object is ***`conn`***:
1. Queries: You can run your queries using ***```psql.read_sql("""<YOUR SQL>""", conn)```***.
2. Create/Delete/Updates: You can run these statements using ***```psql.execute("""<YOUR SQL>""", conn)```***, followed by a ***```conn.commit()```*** command to ensure your transaction is committed. Otherwise your changes will be rolledback if you terminate your kernel.

If you created a new connection object (say to connect to a new cluster) as shown in the last section of `00_database_connectivity_setup.ipynb` notebook, use that connection object where needed.

## 1. Create your schema

In [2]:
sql = """
    create schema YOUR_SCHEMA;
"""
psql.execute(sql, conn)
conn.commit()

DatabaseError: Execution failed on sql '
    create schema YOUR_SCHEMA;
': schema "your_schema" already exists


## 2. Load the Twenty News Groups dataset into a database table

In [8]:
sql = """
    -- Define external table to fetch data from HDFS
    drop external table if exists YOUR_SCHEMA.twenty_news_groups_ext cascade;
    create external table YOUR_SCHEMA.twenty_news_groups_ext
    (
        doc_id int,
        contents text,
        label text
    ) location ('file://gpdb-sandbox.localdomain/home/gpadmin/data/20_newsgroups.tsv') 
    format 'CSV' (DELIMITER = E'\t')
    LOG ERRORS INTO YOUR_SCHEMA.twenty_news_groups_err
    SEGMENT REJECT LIMIT 100;

    -- create an internal table
    drop table if exists YOUR_SCHEMA.twenty_news_groups cascade;
    create table YOUR_SCHEMA.twenty_news_groups
    as
    (
        select 
            *
        from
            YOUR_SCHEMA.twenty_news_groups_ext
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()

In [2]:
sql = """
    select 
        *
    from
       YOUR_SCHEMA.twenty_news_groups
    order by random()
    limit 10
"""
df = psql.read_sql(sql, conn)
conn.commit()
df.head()

,doc_id,contents,label
0,2156,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,comp.os.ms-windows.misc
1,4385,Newsgroups: comp.sys.mac.hardware\nPath: canta...,comp.sys.mac.hardware
2,12219,Newsgroups: sci.electronics\nPath: cantaloupe....,sci.electronics
3,3881,Xref: cantaloupe.srv.cs.cmu.edu comp.sys.ibm.p...,comp.sys.ibm.pc.hardware
4,7870,Newsgroups: rec.autos\nPath: cantaloupe.srv.cs...,rec.autos


## 3. Tokenize the documents using a simple white-space tokenizer

In [11]:
sql = """
    --1) Create inverted index, compute term frequencies
    drop table if exists YOUR_SCHEMA.twenty_news_groups_term_frequencies cascade;
    create table YOUR_SCHEMA.twenty_news_groups_term_frequencies
    as
    (
        select
            token,
            doc_id,
            count(*) as tf
        from
        (
            select
                doc_id,
                regexp_split_to_table(
                    -- replace non-alpha characters, except spaces
                    regexp_replace(
                        --convert to lower case
                        --replace newlines/carriage returns to space
                        regexp_replace(lower(contents), E'\\r|\\n', ' ', 'g'),
                        E'[^a-zA-Z0-9_ ]', '', 'g'
                    ),
                    E'\\\s+'
                ) as token
            from
                YOUR_SCHEMA.twenty_news_groups
        )q
        group by token, doc_id    
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()
df = psql.read_sql("""select * from YOUR_SCHEMA.twenty_news_groups_term_frequencies limit 10;""", conn)
df.head()

,token,doc_id,tf
0,games,10128,1
1,selling,6728,1
2,of,1163,3
3,the,3637,10
4,takes,933,2


## 4. Remove stopwords 

In [12]:
sql = """
    -- Remove stop words 
    drop table if exists YOUR_SCHEMA.twenty_news_groups_vocabulary cascade;
    create table YOUR_SCHEMA.twenty_news_groups_vocabulary
    as
    (
        select
            token
        from
        (
            select
                token,
                count(distinct doc_id) as num_docs
            from
                YOUR_SCHEMA.twenty_news_groups_term_frequencies
            where
                -- Remove stopwords
                token not in (
                    'a\\'s','able','about','above','according','accordingly','across','actually','after','afterwards','again','against','ain\\'t',
                    'all','allow','allows','almost','alone','along','already','also','although','always','am','among','amongst','an','and',
                    'another','any','anybody','anyhow','anyone','anything','anyway','anyways','anywhere','apart','appear','appreciate',
                    'appropriate','are','aren\\'t','around','as','aside','ask','asking','associated','at','available','away','awfully',
                    'be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','believe','below',
                    'beside','besides','best','better','between','beyond','both','brief','but','by','c\\'mon','c\\'s','came','can','can\\'t',
                    'cannot','cant','cause','causes','certain','certainly','changes','clearly','co','com','come','comes','concerning','consequently',
                    'consider','considering','contain','containing','contains','corresponding','could','couldn\\'t','course','currently','definitely',
                    'described','despite','did','didn\\'t','different','do','does','doesn\\'t','doing','don\\'t','done','down','downwards','during',
                    'each','edu','eg','eight','either','else','elsewhere','enough','entirely','especially','et','etc','even','ever','every',
                    'everybody','everyone','everything','everywhere','ex','exactly','example','except','far','few','fifth','first','five',
                    'followed','following','follows','for','former','formerly','forth','four','from','further','furthermore','get','gets',
                    'getting','given','gives','go','goes','going','gone','got','gotten','greetings','had','hadn\\'t','happens','hardly','has',
                    'hasn\\'t','have','haven\\'t','having','he','he\\'s','hello','help','hence','her','here','here\\'s','hereafter','hereby','herein',
                    'hereupon','hers','herself','hi','him','himself','his','hither','hopefully','how','howbeit','however','i\\'d','i\\'ll','i\\'m',
                    'i\\'ve','ie','if','ignored','immediate','in','inasmuch','inc','indeed','indicate','indicated','indicates','inner','insofar',
                    'instead','into','inward','is','isn\\'t','it','it\\'d','it\\'ll','it\\'s','its','itself','just','keep','keeps','kept','know','knows',
                    'known','last','lately','later','latter','latterly','least','less','lest','let','let\\'s','like','liked','likely','little','look',
                    'looking','looks','ltd','mainly','many','may','maybe','me','mean','meanwhile','merely','might','more','moreover','most','mostly',
                    'much','must','my','myself','name','namely','nd','near','nearly','necessary','need','needs','neither','never','nevertheless','new',
                    'next','nine','no','nobody','non','none','noone','nor','normally','not','nothing','novel','now','nowhere','obviously','of','off',
                    'often','oh','ok','okay','old','on','once','one','ones','only','onto','or','other','others','otherwise','ought','our','ours',
                    'ourselves','out','outside','over','overall','own','particular','particularly','per','perhaps','placed','please','plus','possible',
                    'presumably','probably','provides','que','quite','qv','rather','rd','re','really','reasonably','regarding','regardless','regards',
                    'relatively','respectively','right','said','same','saw','say','saying','says','second','secondly','see','seeing','seem','seemed',
                    'seeming','seems','seen','self','selves','sensible','sent','serious','seriously','seven','several','shall','she','should',
                    'shouldn\\'t','since','six','so','some','somebody','somehow','someone','something','sometime','sometimes','somewhat','somewhere',
                    'soon','sorry','specified','specify','specifying','still','sub','such','sup','sure','t\\'s','take','taken','tell','tends','th',
                    'than','thank','thanks','thanx','that','that\\'s','thats','the','their','theirs','them','themselves','then','thence','there',
                    'there\\'s','thereafter','thereby','therefore','therein','theres','thereupon','these','they','they\\'d','they\\'ll','they\\'re',
                    'they\\'ve','think','third','this','thorough','thoroughly','those','though','three','through','throughout','thru','thus','to',
                    'together','too','took','toward','towards','tried','tries','truly','try','trying','twice','two','un','under','unfortunately',
                    'unless','unlikely','until','unto','up','upon','us','use','used','useful','uses','using','usually','value','various','very',
                    'via','viz','vs','want','wants','was','wasn\\'t','way','we','we\\'d','we\\'ll','we\\'re','we\\'ve','welcome','well','went','were',
                    'weren\\'t','what','what\\'s','whatever','when','whence','whenever','where','where\\'s','whereafter','whereas','whereby','wherein',
                    'whereupon','wherever','whether','which','while','whither','who','who\\'s','whoever','whole','whom','whose','why','will',
                    'willing','wish','with','within','without','won\\'t','wonder','would','would','wouldn\\'t','yes','yet','you','you\\'d','you\\'ll',
                    'you\\'re','you\\'ve','your','yours','yourself','yourselves','zero'
                )
            group by token
        )t1,
        (
            select
                count(distinct doc_id) as corpus_size
            from
                YOUR_SCHEMA.twenty_news_groups_term_frequencies
        ) t2
        where 
            -- Only consider those tokens which have occurred in at least 2% of the documents
            t1.num_docs >= 0.02*(t2.corpus_size)
    ) distributed randomly;
"""
psql.execute(sql, conn)
conn.commit()

In [14]:
#Print the vocabulary size
sql = """
    select
        count(*) as vocabulary_size
    from
        YOUR_SCHEMA.twenty_news_groups_vocabulary
"""
df = psql.read_sql(sql, conn)
vocabulary_size = df['vocabulary_size'][0]
display(df)
display(psql.read_sql("select * from YOUR_SCHEMA.twenty_news_groups_vocabulary limit 10", conn))
conn.commit()

,vocabulary_size
0,786


,token
0,10
1,12
2,16
3,1992
4,23
5,27
6,3
7,32
8,5
9,7


## 5. Stem the tokens using the PorterStemmer

In many NLP tasks including document classification and information retrieval, it is useful to work with the root form of words in documents than their derived or inflected forms. Truncating words into their root forms is called stemming. MADlib has an implementation of the classic [Porter Stemmer](http://madlib.incubator.apache.org/docs/latest/group__grp__stemmer.html). Let's try it out on a test sentence below.

In [7]:
sql = """
    select
        madlib_1_10.stem_token_arr(tokens)
    from
    (
        select
            regexp_split_to_array(
                'My mother says that running is good for your health',
                E'\\\s+'
            ) as tokens
    )q
"""
df = psql.read_sql(sql, conn)
display(df)

,stem_token_arr
0,"[My, mother, say, that, run, is, good, for, yo..."
